In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [4]:
import numpy as np
import pandas as pd
import requests
import sqlite3
import os

In [5]:
##get bulk card data
files = requests.get('https://api.scryfall.com/bulk-data').json()
card_data_url = files['data'][0]['download_uri']
card_data = requests.get(card_data_url).json()
card_data[420]

{'object': 'card',
 'id': 'e72daa68-0680-431c-a616-b3693fd58813',
 'oracle_id': '02e1a471-36b2-4d28-a323-f4ec3d095cb9',
 'multiverse_ids': [],
 'name': 'Assembly-Worker',
 'lang': 'en',
 'released_at': '2021-03-19',
 'uri': 'https://api.scryfall.com/cards/e72daa68-0680-431c-a616-b3693fd58813',
 'scryfall_uri': 'https://scryfall.com/card/ttsr/14/assembly-worker?utm_source=api',
 'layout': 'token',
 'highres_image': True,
 'image_status': 'highres_scan',
 'image_uris': {'small': 'https://cards.scryfall.io/small/front/e/7/e72daa68-0680-431c-a616-b3693fd58813.jpg?1641306045',
  'normal': 'https://cards.scryfall.io/normal/front/e/7/e72daa68-0680-431c-a616-b3693fd58813.jpg?1641306045',
  'large': 'https://cards.scryfall.io/large/front/e/7/e72daa68-0680-431c-a616-b3693fd58813.jpg?1641306045',
  'png': 'https://cards.scryfall.io/png/front/e/7/e72daa68-0680-431c-a616-b3693fd58813.png?1641306045',
  'art_crop': 'https://cards.scryfall.io/art_crop/front/e/7/e72daa68-0680-431c-a616-b3693fd58813.jp

In [8]:
# Create the card_db folder if it doesn't exist

os.makedirs('card_db', exist_ok=True)

# Connect to the SQLite database
db_path = os.path.join('card_db', 'cards.db')
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Create a table for the card data
cursor.execute('''
CREATE TABLE IF NOT EXISTS cards (
    id TEXT PRIMARY KEY,
    name TEXT,
    mana_cost TEXT,
    cmc REAL,
    type_line TEXT,
    oracle_text TEXT,
    power TEXT,
    toughness TEXT,
    colors TEXT,
    color_identity TEXT,
    keywords TEXT,
    released_at TEXT,
    scryfall_uri TEXT
)
''')

# Insert card data into the database
for card in card_data:
    cursor.execute('''
    INSERT OR REPLACE INTO cards (
        id, name, mana_cost, cmc, type_line, oracle_text, power, toughness, 
        colors, color_identity, keywords, released_at, scryfall_uri
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (
        card.get('id'),
        card.get('name'),
        card.get('mana_cost'),
        card.get('cmc'),
        card.get('type_line'),
        card.get('oracle_text'),
        card.get('power'),
        card.get('toughness'),
        ','.join(card.get('colors', [])),
        ','.join(card.get('color_identity', [])),
        ','.join(card.get('keywords', [])),
        card.get('released_at'),
        card.get('scryfall_uri')
    ))

# Commit changes and close the connection
conn.commit()

print(f"Database created at {db_path}")

Database created at card_db\cards.db


In [10]:
cards = pd.read_sql_query("SELECT * FROM cards", conn)
cards.sample(10)


,id,name,mana_cost,cmc,type_line,oracle_text,power,toughness,colors,color_identity,keywords,released_at,scryfall_uri
27541,822d0f73-cfb0-41d9-b4eb-09c605112a13,Undercity Informer,{2}{B},3.0,Creature — Human Rogue,"{1}, Sacrifice a creature: Target player revea...",2,3,B,B,,2013-02-01,https://scryfall.com/card/gtc/82/undercity-inf...
4546,6a1d7d4b-721e-4946-803d-b04ddbe7b718,Sea Monster,{4}{U}{U},6.0,Creature — Serpent,This creature can't attack unless defending pl...,6,6,U,U,,2015-05-06,https://scryfall.com/card/tpr/65/sea-monster?u...
12463,d6ffeab4-83b1-4414-ae72-e59a2354ea15,Homarid,{2}{U},3.0,Creature — Homarid,This creature enters with a tide counter on it...,2,2,U,U,,1994-11-01,https://scryfall.com/card/fem/19a/homarid?utm_...
4857,42f01f4d-491e-4f25-a017-a0d9c437ac8c,Blessing of Frost,{3}{G},4.0,Snow Sorcery,Distribute X +1/+1 counters among any number o...,None,None,G,G,,2021-02-05,https://scryfall.com/card/khm/161/blessing-of-...
27374,24bbdeb0-9165-4874-a853-d19c20c250ef,Lucky Offering,{W},1.0,Sorcery,Destroy target artifact with mana value 3 or l...,None,None,W,W,,2022-02-18,https://scryfall.com/card/neo/27/lucky-offerin...
244,afb8ebe9-8648-4bd1-b725-e0d9656f53e3,You Compleat Me,{1}{B}{B},3.0,Sorcery,"If your life total is greater than 10, it beco...",None,None,B,B,,2024-08-02,https://scryfall.com/card/mb2/556/you-compleat...
16349,2c1392c5-91a5-4e6e-803d-ed032e4d594b,A Killer Among Us,{4}{G},5.0,Enchantment,"When this enchantment enters, create a 1/1 whi...",None,None,G,G,,2024-02-09,https://scryfall.com/card/mkm/167/a-killer-amo...
17818,f5a39379-7313-463a-9a02-e5157b9557f4,Angrath's Rampage,{B}{R},2.0,Sorcery,Choose one —\n• Target player sacrifices an ar...,None,None,"B,R","B,R",,2019-05-03,https://scryfall.com/card/war/185/angraths-ram...
30693,2a1ec927-b645-4285-8765-02c0ac104ae1,Rolling Temblor,{2}{R},3.0,Sorcery,Rolling Temblor deals 2 damage to each creatur...,None,None,R,R,Flashback,2019-08-23,https://scryfall.com/card/c19/151/rolling-temb...
11568,08db5b31-6b5a-4b99-b103-d628afaea2ba,O-Naginata,{1},1.0,Artifact — Equipment,This Equipment can be attached only to a creat...,None,None,,,Equip,2020-08-07,https://scryfall.com/card/2xm/278/o-naginata?u...
